In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from datetime import datetime
import glob
import re
import warnings
warnings.filterwarnings('ignore')

In [ ]:
drive.mount('/content/drive', force_remount = True)
%cd /content/drive/My Drive/Colab Notebooks/Aniish/Aniish/Earning

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/Aniish/Aniish/Earning


In [ ]:
def extract_text(file_txt_path):
    lines = []
    with open(file_txt_path) as f:

      for line in f:
        lines.append(line.strip())
        if "Questions and Answers" in line:
          break
    return lines

def check_lines(string_sentence):

  if "Operator" in string_sentence:
    return False

  if "-" and "," and '[' and ']' in string_sentence:
    return True

  else:
    return False

def extract_speaker_index(lst_sentences):

  sent_index = []

  for count, values in enumerate(lst_sentences):
    if check_lines(values) == True:
      sent_index.append(count)

  return sent_index

def extract_year_date_info(file_txt_path):

  sentences = extract_text(file_txt_path)
  ticker = file_txt_path.split("-")[3].split(".")[0]
  date_val = sentences[6].split("/")[0].strip()
  quarter_val = sentences[5][0:7]
  time_stamp = sentences[6].split("/")[1].strip()


  return [sentences, ticker, date_val, quarter_val, time_stamp]







In [ ]:
def extract_sentences(file_path):


  df_container = pd.DataFrame({
      "Ticker" : [],
      "Date_Val": [],
      "Quarter_Val":[],
      "Time_Stamp":[],
      "Individual" : [],
      "Sentence": []}, index = [])


  lst_extract_all = extract_year_date_info(file_path)

  lst_sentences = lst_extract_all[0]
  lst_index = extract_speaker_index(lst_sentences)

  if lst_index == []:
    return df_container
  else:
    pass

  speaker_name = ''
  speaker_desg = ''
  start_index = 0
  end_index = 0

  ticker_val = lst_extract_all[1]
  date_val = lst_extract_all[2]
  quarter_val = lst_extract_all[3]
  time_stamp = lst_extract_all[4]


  for counter, values in enumerate(lst_index):

    speaker_name = lst_sentences[values]
    start_index = values + 3
    end_index = lst_index[counter + 1] - 2 if counter + 1 < len(lst_index) else -1

    sentences_current_speaker = (lst_sentences[start_index : end_index])


    for sentences in sentences_current_speaker:
      sentences = sentences.replace(".com", "com" ) if ".com" in sentences else sentences
      lines = sentences.split(".")
      lines = list(filter(lambda x: x.strip(), lines))

      for line in lines:
        df_current_speaker_content = pd.DataFrame({"Ticker" : [ticker_val],
                                                   "Date_Val": [date_val],
                                                   "Quarter_Val":[quarter_val],
                               "Time_Stamp":[time_stamp],
                              "Individual" : [speaker_name],
                              "Sentence": [line]})
        df_container = pd.concat([df_container, df_current_speaker_content])

  df_container = df_container.reset_index()

  df_container = df_container[df_container['Sentence'].str.contains("Good Morning|Thank you") == False]
  df_container = df_container.drop('index', axis = 1)

  return df_container



In [ ]:

df_full_data =  pd.DataFrame({"Ticker" : [],
                                "Date_Val": [],
                                "Quarter_Val":[],
                               "Time_Stamp":[],
                              "Individual" : [],
                              "Sentence": []}, index = [])

for file in glob.glob("*.txt"):
    print(file)
    df_current_file = extract_sentences(file)
    df_full_data = pd.concat([df_full_data, df_current_file], axis = 0)


Streaming output truncated to the last 5000 lines.
2022-Jul-27-MSFT.OQ-139232561311-Transcript.txt
2018-Apr-17-UNH.N-139876094901-Transcript.txt
2013-Feb-06-DIS.N-138497946513-Transcript.txt
2024-Feb-07-AMGN.OQ-139441030000-Transcript.txt
2016-Jan-28-LLY.N-139753177768-Transcript.txt
2021-Jul-28-MSFT.OQ-138684941105-Transcript.txt
2022-Jan-26-MSFT.OQ-140806504057-Transcript.txt
2016-Jul-26-LLY.N-138043614001-Transcript.txt
2023-Apr-27-AMGN.OQ-140318789876-Transcript.txt
2018-Nov-06-LLY.N-138855989064-Transcript.txt
2017-Jan-26-QCOM.OQ-138968519994-Transcript.txt
2014-Aug-06-DIS.N-139239361723-Transcript.txt
2015-Jul-23-LLY.N-138651090724-Transcript.txt
2023-Oct-25-MSFT.OQ-139993350133-Transcript.txt
2016-Jul-20-QCOM.OQ-141023747142-Transcript.txt
2018-Oct-16-UNH.N-140456086254-Transcript.txt
2018-Jul-17-UNH.N-140044726305-Transcript.txt
2019-Apr-16-UNH.N-138046669003-Transcript.txt
2023-Oct-31-AMGN.OQ-141001902840-Transcript.txt
2024-Jan-31-MSFT.OQ-139756114312-Transcript.txt
2024-Oct-

In [ ]:
df_full_data['Name'] = df_full_data['Individual'].apply(lambda x: x.split(',')[0])
df_full_data['Designation'] = df_full_data['Individual'].apply(lambda x: ':'.join(x.split(',')[1:]) if len(x.split(',')) >=2 else ' ')

df_full_data['Quarter'] = df_full_data['Quarter_Val'].apply(lambda x: x.split(' ')[0])
df_full_data['Year'] = df_full_data['Quarter_Val'].apply(lambda x: x.split(' ')[1] if len(x.split(' ')) >=2 else ' ')

In [ ]:
df_data_final = df_full_data[['Ticker', 'Date_Val', 'Time_Stamp', 'Quarter', 'Year', 'Name', 'Designation', 'Sentence', 'Individual', 'Quarter_Val']]

In [ ]:
df_data_final.to_csv("All_Data.csv")